# Bài 4: Pandas (part 2)

In [ ]:
import pandas as pd
import numpy as np

## 3. Data frames

- Data frame thể hiện cấu trúc dạng bảng như excel sheet
- Có rows và columns
- Mỗi column là một Series

### 3.1. Khởi tạo

- Từ dictionary

In [ ]:
d = {
    "name": ["John", "Bob", "Jane"],
    "age": [18, 20, 30],
    "edu": ["BS", "MS", "BS"]
}

df = pd.DataFrame(d)
df

- Từ 2d-list

In [ ]:
l = [
    ("John", 18, "BS"),
    ("Bob", 20, "MS"),
    ("Jane", 30, "BS")
]

df = pd.DataFrame(l, columns=["name", "age", "edu"])
df

### 3.2. Basic operations

#### A) Input
- Dùng `pd.read_xxx`. VD: `read_csv`, `read_excel`, ...

In [ ]:
df = pd.read_excel("data/invoice_2011_07.xlsx")

#### B) Inpsect

- View first few rows

In [ ]:
df.head(2)

- View last few rows

In [ ]:
df.tail(2)

- Num rows and num cols

In [ ]:
# Shape
df.shape

In [ ]:
# Num rows
df.shape[0]

In [ ]:
# Num cols
df.shape[1]

- Column names

In [ ]:
df.columns.tolist()

- Dtype của từng cột

In [ ]:
df.dtypes

- Transpose đổi hàng thành cột (for better view)

In [ ]:
df.head().T

#### C) Thao tác với cột

- Truy xuất 1 cột -> trả về 1 series

In [ ]:
df["InvoiceNo"]

- Truy xuất nhiều cột -> trả về data frame

In [ ]:
df[["InvoiceNo", "UnitPrice", "Quantity"]].head()

- Chọn nhiều cột, dùng `.loc` (recommended)

In [ ]:
df.loc[:, ["InvoiceNo", "UnitPrice", "Quantity"]].head()

- Chọn nhiều cột liên tiếp

In [ ]:
df.loc[:, "Quantity":"Country"].head()

- Update cột có sẵn

In [ ]:
# Giả sử quantity được lưu dưới đơn vị 100
# Giá trị 6 nghĩa là 6000
# Update lại cột này cho đúng giá trị
df["Quantity"] = df["Quantity"] * 1000
df.head(2)

In [ ]:
# Update lại cột InvoiceDate về dạng datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.dtypes

- Thêm cột mới

In [ ]:
# Thêm cột Revenue = Quantity * Unit price
df["Revenue"] = df["Quantity"] * df["UnitPrice"]
df.head(2)

In [ ]:
# Thêm cột mới từ một literal
# Giả sử file này đến từ store 1, có thể sẽ được combined với các file từ store khác
# Tạo 1 cột mới để đánh dấu các đơn hàng này đến từ store nào
df["Store"] = "store 1"
df.head(2)

- Các thao tác với cột kiểu `datetime`

In [ ]:
# Thêm cột Year 
df["Year"] = df["InvoiceDate"].dt.year
df.head(2)

In [ ]:
# Thêm cột Month 
df["Month"] = df["InvoiceDate"].dt.month
df.head(2)

In [ ]:
# Thêm cột Date 
df["Date"] = df["InvoiceDate"].dt.day
df.head(2)

- Xóa cột

In [ ]:
# Xóa cột Store vừa tạo dùng del
del df["Store"]
df.head(2)

In [ ]:
# Xóa cột Revenue dùng drop (nhớ set inplace=True nếu muốn lưu thay đổi)
# Cách này dùng xóa nhiều cột rất tiện
df.drop(columns=["Revenue"], inplace=True)
df.head(2)

- Unique values

In [ ]:
# Đếm số unique countries
df["Country"].nunique()

In [ ]:
# Lấy ra danh sách unique countries
df["Country"].unique().tolist()

In [ ]:
# View distribution của unique countries - theo absolute count
df["Country"].value_counts()

In [ ]:
# View distribution của unique countries - theo percentage
df["Country"].value_counts(normalize=True) * 100

- Các thao tác với cột dạng string

In [ ]:
# Upper cột Country
df["Country"].str.upper()

In [ ]:
# Lower cột Country
df["Country"].str.lower()

In [ ]:
# Chaining
df["Country"].str.strip().str.upper()

In [ ]:
# Lọc ra những item có description chứa từ 'cake' (ko quan tâm hoa thường)
# Note: use comparison
cond = df["Description"].str.lower().str.contains("cake") == True
df["Description"].loc[cond]

#### D) Thao tác với hàng (rows)

- Trước hết, inspect lại `df`

In [ ]:
df.head(2)

In [ ]:
df["Date"].unique().tolist()

- Lọc ra tất cả các hóa đơn trong ngày 10 đến 20 của tháng 7 năm 2011
- Có bao nhiêu dòng như vậy?

In [ ]:
# Cách 1
df2 = df.loc[(df["Year"] == 2011) & (df["Month"] == 7) & (df["Date"] >= 10) & (df["Date"] <= 20), :]
df2.shape

In [ ]:
# Cách 2
cond = (df["Year"] == 2011) & \
    (df["Month"] == 7) & \
    (df["Date"] >= 10) & \
    (df["Date"] <= 20)

df2 = df.loc[cond, :]
df2.shape

In [ ]:
# Cách 3
cond = (df["InvoiceDate"] >= '2011-07-10') & (df["InvoiceDate"] < '2011-07-21' )
df2 = df.loc[cond, :]
df2.shape

- Lọc ra những sản phẩm description có chứa `'STRAWBERY'`

In [ ]:
cond = df["Description"].str.upper().str.contains("STRAWBERY") == True
df2 = df.loc[cond, :]
df2.shape

- Có bao nhiêu hóa đơn không có thông tin khách hàng

In [ ]:
df2 = df.loc[df["CustomerID"].isnull(), :]
df2.shape

- Có bao nhiêu hóa đơn có thông tin khách hàng

In [ ]:
df2 = df.loc[df["CustomerID"].notnull(), :]
df2.shape

#### E) Thao tác hàng và cột cùng lúc

- Lọc ra những đơn hàng có quantity `>= 1,000,000` và chỉ giữ lại cột `InvoiceDate`, `Quantity`, `UnitPrice`

In [ ]:
# Cách 1: break down
cond = df["Quantity"] >= 1e6
cols = ["InvoiceDate", "Quantity", "UnitPrice"]

df.loc[cond, cols]

In [ ]:
# Cách 2: Viết gộp
df.loc[df["Quantity"] >= 1e6, ["Quantity", "UnitPrice"]]